In [13]:
import pandas as pd
import datetime
df = pd.read_csv('locust_data.csv')

df.columns.unique()

Index(['Category', 'lat', 'lon', 'Part of month', 'Observation Date',
       'Impacted Area (ha)', 'Country', 'Location Name', 'Data updated on'],
      dtype='object')

In [14]:
df['Observation Date'] = pd.to_datetime(df['Observation Date'], errors='coerce')
df_2018_onwards = df[df['Observation Date'] >= '2018-01-01']

df_2018_onwards.to_csv('locust_data_2018_onwards.csv', index=False)


PermissionError: [Errno 13] Permission denied: 'locust_data_2018_onwards.csv'

In [15]:
df = pd.read_csv("locust_data_2018_onwards.csv")

In [16]:
df

,Category,lat,lon,Part of month,Observation Date,Impacted Area (ha),Country,Location Name,Data updated on
0,ControlREV,11.339167,42.768611,Middle,2021-06-15,1.0,Djibouti,Arta,Tue Feb 1 16:04:44 2022
1,Swarm,31.666667,44.600000,Early,2020-04-01,45.0,Iraq,Al Najaf,Tue Feb 1 16:04:44 2022
2,Band,11.228889,42.798333,Middle,2021-06-18,3.0,Djibouti,Ali Sabieh,Tue Feb 1 16:04:44 2022
3,Adult,17.046667,20.947778,Middle,2020-10-14,20.0,Chad,SALLAH 6,Tue Feb 1 16:04:44 2022
4,Adult,13.793611,19.176667,Late,2020-10-21,20.0,Chad,Kachako,Tue Feb 1 16:04:44 2022
...,...,...,...,...,...,...,...,...,...
17851,Adult,17.266944,21.648333,Middle,2018-11-17,30.0,Democratic Republic of the Congo,Delinie,Tue Feb 1 16:04:44 2022
17852,Adult,15.941944,20.931111,Late,2019-08-22,20.0,Democratic Republic of the Congo,Kowoulang,Tue Feb 1 16:04:44 2022
17853,Adult,16.735278,21.893056,Middle,2019-09-11,20.0,Democratic Republic of the Congo,Toukou,Tue Feb 1 16:04:44 2022
17854,Adult,16.574444,23.041111,Middle,2019-09-17,20.0,Democratic Republic of the Congo,Oroki,Tue Feb 1 16:04:44 2022


In [17]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

url = "https://archive-api.open-meteo.com/v1/archive"

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Sample dataset (assuming you have already loaded df with 'lat', 'lon', 'Observation Date' columns)
# df = pd.read_csv('your_data.csv')  # Uncomment to load your data

# Prepare lists to store the results
temperatures = []
soil_moistures = []

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    latitude = row['lat']
    longitude = row['lon']
    observation_date = pd.to_datetime(row['Observation Date']).strftime('%Y-%m-%d')
    
    # Define parameters for the API request
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": observation_date,
        "end_date": observation_date,
        "daily": ["temperature_2m_max"], 
        "hourly": ["soil_moisture_0_to_1cm"]
    }
    
    # Make the API request
    responses = openmeteo.weather_api(url, params=params)
    
    # Extract data from the response
    response = responses[0]
    hourly_data = response.HourlyDataFrame()
    
    # Find the data for 12:00 PM (assuming timezone handling matches your input data)
    temperature_12pm = hourly_data.loc[hourly_data.index.hour == 12, 'temperature_2m'].iloc[0]
    soil_moisture_12pm = hourly_data.loc[hourly_data.index.hour == 12, 'soil_moisture_0_1cm'].iloc[0]
    
    # Append results to the lists
    temperatures.append(temperature_12pm)
    soil_moistures.append(soil_moisture_12pm)

# Add the results to the DataFrame
df['Temperature_12PM'] = temperatures
df['Soil_Moisture_12PM'] = soil_moistures

# Print the updated DataFrame or save it
print(df.head())


AttributeError: 'WeatherApiResponse' object has no attribute 'HourlyDataFrame'